In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import os

Mann Whytney U test for survival rates per generations

In [2]:
def survival_rate(df):
    generations = df['Generation'].unique()
    generations.sort()    
    operator_survival_rate = {}
    
    for operator in df['Operator'].unique():
        y = []
        subset = df[df['Operator'] == operator]
        for i in range(len(generations)):
            subset_generation = subset[subset['Generation'] == i]
            y.append(len(subset_generation[subset_generation['Passed'] == 1])/400)
            operator_survival_rate[operator] = y
       
    operator_survival_df = pd.DataFrame(operator_survival_rate)
    operators = list(operator_survival_rate.keys())
    comparison_results = []
        
    for i in range(len(operators)):
        for j in range(i + 1, len(operators)):
            op1, op2 = operators[i], operators[j]
            stat, p = compare_operators(operator_survival_df, op1, op2)
            comparison_results.append((op1, op2, p))

    comparison_results_df = pd.DataFrame(comparison_results, columns=["Operator1", "Operator2", "p-value"])

    significance_level = 0.05
    comparison_results_df['Significant'] = comparison_results_df['p-value'] < significance_level
    results = comparison_results_df[comparison_results_df['Significant'] == True]
    
    points = {x:0 for x in operators}
    
    for index, row in results.iterrows():
        operator1 = row['Operator1']
        operator2 = row['Operator2']
        
        points[operator1] += np.mean(operator_survival_rate[operator1]) - np.mean(operator_survival_rate[operator2])
        points[operator2] += np.mean(operator_survival_rate[operator2]) - np.mean(operator_survival_rate[operator1])
        
    return points
    
def compare_operators(df, operator1, operator2):
    data1 = df[operator1].dropna()
    data2 = df[operator2].dropna()
    stat, p = stats.mannwhitneyu(data1, data2, alternative='two-sided')
    return stat, p


In order to print the results, we need to run the following code:

In [3]:

# Write the folder with the dataframes. 
# The dataframes should be in the format of the ones saved by the learn_ft.py script
# This code will calculate the mean and confidence interval of the scores by using all the dataframes in the folder.
# The default folder, where the required data frames for this test are saved, is: saved_data_frames/difference
folder_path = '..\\saved_data_frames\\difference\\COVID_19'

file_list = os.listdir(folder_path)
all_scores = []
filename = os.path.basename(os.path.normpath(folder_path))

for file_name in file_list:
    if file_name.endswith('.pkl'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_pickle(file_path)
        all_scores.append(survival_rate(df))

    
agg_scores = {x : [] for x in all_scores[0].keys()}
        
for operator in all_scores[0].keys():
    for score in all_scores:
        agg_scores[operator].append(score[operator])
        
result = {}
for operator in agg_scores.keys():
    values = agg_scores[operator]
    mean = np.mean(values)
    
    std_dev = np.std(values)

    z_score = stats.norm.ppf(0.975)
    n = len(values)
    margin_of_error = z_score * (std_dev / np.sqrt(n))

    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error
    
    result[operator] = [mean, lower_bound, upper_bound]

df = pd.DataFrame.from_dict(result, orient = 'index', columns =['Mean', 'Lower Bound', 'Upper Bound'])
df = df.sort_values(by='Mean', ascending=False)
print(df)     


                      Mean  Lower Bound  Upper Bound
disconnect_be     0.409423     0.386036     0.432809
delete_be         0.194926     0.149051     0.240801
cross_over        0.089725     0.071552     0.107899
create_be         0.072999     0.032185     0.113814
delete_gate       0.029146    -0.008410     0.066703
move_be          -0.076216    -0.107476    -0.044957
connect_be       -0.214841    -0.233957    -0.195725
create_gate      -0.237405    -0.245444    -0.229367
change_gate_type -0.267757    -0.289138    -0.246376
